In [1]:
import numpy as np

In [64]:
def bitboard():
    return np.zeros((8,8), dtype=bool)

def giveBitboards():
    # creates bitboards
    b = {
        
        # lines
        "la" : bitboard(),
        "lb" : bitboard(),
        "lc" : bitboard(),
        "ld" : bitboard(),
        "le" : bitboard(),
        "lf" : bitboard(),
        "lg" : bitboard(),
        "lh" : bitboard(),
        # rows
        "1" : bitboard(),
        "2" : bitboard(),
        "3" : bitboard(),
        "4" : bitboard(),
        "5" : bitboard(),
        "6" : bitboard(),
        "7" : bitboard(),
        "8" : bitboard(),
        # piece colors
        "W" : bitboard(),
        "B" : bitboard(),
        # pieces
        "k" : bitboard(),
        "q" : bitboard(),
        "n" : bitboard(),
        "r" : bitboard(),
        "b" : bitboard(),
        "p" : bitboard(),
        
    }
    return b

def initGame(b):
    # initializes board
    b['W'][0:2,:] = True
    b['B'][-2:,:] = True
    
    
    
    b['k'][[0,-1],[4,4]] = True
    b['q'][[0,-1],[3,3]] = True
    b['n'][[0,0,-1,-1],[1,-2,1,-2]] = True
    b['r'][[0,0,-1,-1],[0,-1,0,-1]] = True
    b['b'][[0,0,-1,-1],[2,-3,2,-3]] = True
    b['p'][[1,-2],:] = True
    
    return b


def printBoard(b):
    # pretty print board
    board = np.empty((8,8), dtype=str)
    board[:] = '_'
    
    board[b['B'] & b['k']] = 'k'
    board[b['B'] & b['q']] = 'q'
    board[b['B'] & b['n']] = 'n'
    board[b['B'] & b['r']] = 'r'
    board[b['B'] & b['b']] = 'b'
    board[b['B'] & b['p']] = 'p'
    
    board[b['W'] & b['k']] = 'K'
    board[b['W'] & b['q']] = 'Q'
    board[b['W'] & b['n']] = 'N'
    board[b['W'] & b['r']] = 'R'
    board[b['W'] & b['b']] = 'B'
    board[b['W'] & b['p']] = 'P'

    cols = np.reshape(['a','b','c','d','e','f','g','h'],(8,1))
    rows = np.reshape(['/','1','2','3','4','5','6','7','8'],(9,1))
    
    print(board.shape)
    print(cols.shape)
    print(rows.shape)
    
    board = np.hstack((board, np.transpose(cols)))
    return board

In [65]:
def generateMoves(b, player):
    moves = list
    # striking
    
    # moving
    
    
    return moves

def movablePawnsW(b):
    return np.roll((b['W'] & b['p']), 8) & ~(b['W'] | b['B'])


#((B & SCH) >> 8) & ~(WEI | SCH)

In [66]:
b = initGame(giveBitboards())
print(printBoard(b))
movablePawnsW(b)

(8, 8)
(8, 1)
(9, 1)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 8 and the array at index 1 has size 1

In [ ]:
#pieces
starting(pieces)
pieces['q'] & pieces['B'] 